In [13]:
# setup
import numpy as np
import pandas as pd

from scipy.stats import norm, uniform, sem

from utility import *
from theoretical import *

pd.set_option("display.float_format", "{:.4f}".format)

# Chapter 6 - Variance Reduction Techniques

## Antithetic Sampling

**Example 6.1.** Use antithetic sampling to estimate the price of a call option
with maturity $T$ and strike price $K$. Compare with the plain Monte Carlo
estimate.

In [14]:
def example_6_1():
    rng = np.random.default_rng(4198)
    
    # simulation params
    S_0, r, sig, T = 50, 0.05, 0.2, 1
    
    def antithetic(n, K):
        # antithetic sampling
        Z = norm.rvs(0, 1, size=n, random_state=rng)
        
        S = S_0 * np.exp((r - 0.5 * sig**2) * T + sig * np.sqrt(T) * Z)
        X = np.exp(-r * T) * np.maximum(S - K, 0)
        
        S = S_0 * np.exp((r - 0.5 * sig**2) * T - sig * np.sqrt(T) * Z)
        Y = np.exp(-r * T) * np.maximum(S - K, 0)

        # compute mean est. and std. error
        H  = (X + Y) / 2
        v_hat = np.mean(H)
        se = sem(H)

        return v_hat, se
    
    def plain(n, K):
        # sampling
        Z = norm.rvs(0, 1, size=2 * n, random_state=rng)
        Y = S_0 * np.exp((r - 0.5 * sig**2) * T + sig * np.sqrt(T) * Z)
        X = np.exp(-r * T) * np.maximum(Y - K, 0)

        # compute mean est. and std. error
        v_hat = np.mean(X)
        se = sem(X)

        return v_hat, se

    theoretical = {}
    mc_est = {}
    se_vals = {}
    n, strikes, methods = 10_000, [40, 50, 60], ["antithetic", "plain"]
    
    for k in strikes:
        theoretical_val = european_call(S_0, k, r, T, sig) 
        mean_est_plain, std_err_plain = plain(n, k)
        mean_est_antithetic, std_err_antithetic = antithetic(n, k)

        antithetic_key = (f"{k}", "antithetic")
        plain_key = (f"{k}", "plain")

        theoretical[antithetic_key] = theoretical_val
        mc_est[antithetic_key] = mean_est_antithetic
        se_vals[antithetic_key] = std_err_antithetic

        theoretical[plain_key] = ""
        mc_est[plain_key] = mean_est_plain
        se_vals[plain_key] = std_err_plain
    
    # Build MultiIndex columns
    columns = pd.MultiIndex.from_product(
        [strikes, methods],
        names=["Strike price", ""]
    )

    # Collect rows
    rows = {
        "True value": [theoretical[(str(k), m)] for k in strikes for m in methods],
        "Estimate":   [mc_est[(str(k), m)] for k in strikes for m in methods],
        "S.E.":       [se_vals[(str(k), m)] for k in strikes for m in methods],
    }

    # Create DataFrame
    df = pd.DataFrame(rows, index=columns).T

    return df

In [15]:
results_6_1 = example_6_1()
results_6_1

Strike price         40                 50                60       
             antithetic   plain antithetic  plain antithetic  plain
True value      12.2944             5.2253            1.6237       
Estimate        12.3118 12.3228     5.1584 5.2174     1.6181 1.5759
S.E.             0.0236  0.0680     0.0361 0.0517     0.0287 0.0298

**Example 6.2.** Consider a discretely monitored down-and-out barrier option
with maturity $T$ and payoff
$$(S_T − K)^{+} \cdot \boldsymbol{1}_{\left\{\min(S_{t_1} , \cdots ,S_{t_m} ) \geq b\right\}}.$$
The monitoring dates $0 < t_1 < t_2 < \cdots < t_m = T$ are prefixed. Compare
the antithetic sampling estimate with the plain Monte Carlo estimate.

In [33]:
def example_6_2():
    rng = np.random.default_rng(4198)
    
    # simulation params
    S_0, r, sig, T = 50, 0.05, 0.2, 1
    b, m = 45, 12

    t = [i * T / m for i in range(1, m + 1)]
    
    def antithetic(n, K):
        # antithetic sampling
        Z = norm.rvs(0, 1, size=(m, n), random_state=rng)
        St = np.zeros(shape=(m, n))
        St_bar = np.zeros(shape=(m, n))

        t_diff = [t[0]] + [t[i] - t[i-1] for i in range(1, m)]
        
        St[0] = S_0 * np.exp((r - 1/2 * sig**2) * t_diff[0] + sig * np.sqrt(t_diff[0]) * Z[0])
        St_bar[0] = S_0 * np.exp((r - 1/2 * sig**2) * t_diff[0] - sig * np.sqrt(t_diff[0]) * Z[0])
        for i in range(1, m):
            St[i] = St[i-1] * np.exp((r - 1/2 * sig**2) * t_diff[i] + sig * np.sqrt(t_diff[i]) * Z[i])
            St_bar[i] = St_bar[i-1] * np.exp((r - 1/2 * sig**2) * t_diff[i] - sig * np.sqrt(t_diff[i]) * Z[i])
        
        X = np.exp(-r*T) * np.maximum(St[m-1] - K, 0) * (np.min(St, axis=0) >= b)
        Y = np.exp(-r*T) *  np.maximum(St_bar[m-1] - K, 0) * (np.min(St_bar, axis=0) >= b)

        # compute mean est. and std. error
        H  = (X + Y) / 2
        v_hat = np.mean(H)
        se = sem(H)

        return v_hat, se
    
    def plain(n, K):
        # sampling
        Z = norm.rvs(0, 1, size=(m, 2 * n), random_state=rng)
        St = np.zeros(shape=(m, 2 * n))

        t_diff = [t[0]] + [t[i] - t[i-1] for i in range(1, m)]
        
        St[0] = S_0 * np.exp((r - 1/2 * sig**2) * t_diff[0] + sig * np.sqrt(t_diff[0]) * Z[0])
        for i in range(1, m):
            St[i] = St[i-1] * np.exp((r - 1/2 * sig**2) * t_diff[i] + sig * np.sqrt(t_diff[i]) * Z[i])
        
        X = np.exp(-r*T) * np.maximum(St[m-1] - K, 0) * (np.min(St, axis=0) >= b)

        # compute mean est. and std. error
        v_hat = np.mean(X)
        se = sem(X)

        return v_hat, se

    mc_est = {}
    se_vals = {}
    n, strikes, methods = 10_000, [40, 50, 60], ["antithetic", "plain"]
    
    for k in strikes:
        mean_est_plain, std_err_plain = plain(n, k)
        mean_est_antithetic, std_err_antithetic = antithetic(n, k)

        antithetic_key = (f"{k}", "antithetic")
        plain_key = (f"{k}", "plain")

        mc_est[antithetic_key] = mean_est_antithetic
        se_vals[antithetic_key] = std_err_antithetic

        mc_est[plain_key] = mean_est_plain
        se_vals[plain_key] = std_err_plain
    
    # Build MultiIndex columns
    columns = pd.MultiIndex.from_product(
        [strikes, methods],
        names=["Strike price", ""]
    )

    # Collect rows
    rows = {
        "Estimate":   [mc_est[(str(k), m)] for k in strikes for m in methods],
        "S.E.":       [se_vals[(str(k), m)] for k in strikes for m in methods],
    }

    # Create DataFrame
    df = pd.DataFrame(rows, index=columns).T

    return df

In [34]:
results_6_2 = example_6_2()
results_6_2

Strike price         40                50                60       
             antithetic  plain antithetic  plain antithetic  plain
Estimate         9.9252 9.9396     4.7907 4.7763     1.5767 1.6285
S.E.             0.0432 0.0768     0.0397 0.0522     0.0286 0.0314

**Example 6.3.** Use antithetic sampling to estimate the price of a butterfly
spread option with maturity $T$ and payoff
$$(S^T − K_1)^+ + (S^T − K_3)+ − 2(S^T − K_2)^+,$$
where $0 < K_1 < K_3$ and $K_2 = (K_1 + K_3)/2$. Compare with the plain Monte
Carlo estimate.

In [ ]:
def example_6_3():
    rng = np.random.default_rng(4198)
    
    # simulation params
    S_0, r, sig = 50, 0.05, 0.2
    
    K = [45, 50, 55]
    n = 10_000
    
    def antithetic(T):
        # antithetic sampling
        Z = norm.rvs(0, 1, size=n, random_state=rng)
        
        S = S_0 * np.exp((r - 0.5 * sig**2) * T + sig * np.sqrt(T) * Z)
        X = np.exp(-r * T) * (np.maximum(S - K[0], 0) + np.maximum(S - K[2], 0) - 2 * np.maximum(S - K[1], 0))
        
        S = S_0 * np.exp((r - 0.5 * sig**2) * T - sig * np.sqrt(T) * Z)
        Y = np.exp(-r * T) * (np.maximum(S - K[0], 0) + np.maximum(S - K[2], 0) - 2 * np.maximum(S - K[1], 0))

        # compute mean est. and std. error
        H  = (X + Y) / 2
        v_hat = np.mean(H)
        se = sem(H)

        return v_hat, se
    
    def plain(T):
        # sampling
        Z = norm.rvs(0, 1, size=2 * n, random_state=rng)
        S = S_0 * np.exp((r - 0.5 * sig**2) * T + sig * np.sqrt(T) * Z)
        X = np.exp(-r * T) * (np.maximum(S - K[0], 0) + np.maximum(S - K[2], 0) - 2 * np.maximum(S - K[1], 0))

        # compute mean est. and std. error
        v_hat = np.mean(X)
        se = sem(X)

        return v_hat, se

    theoretical = {}
    mc_est = {}
    se_vals = {}
    n, maturation, methods = 10_000, [1, 0.5, 0.25], ["antithetic", "plain"]
    
    for T in maturation:
        theoretical_val = butterfly_spread(S_0, K, r, T, sig) 
        mean_est_plain, std_err_plain = plain(T)
        mean_est_antithetic, std_err_antithetic = antithetic(T)

        antithetic_key = (f"{T}", "antithetic")
        plain_key = (f"{T}", "plain")

        theoretical[antithetic_key] = theoretical_val
        mc_est[antithetic_key] = mean_est_antithetic
        se_vals[antithetic_key] = std_err_antithetic

        theoretical[plain_key] = ""
        mc_est[plain_key] = mean_est_plain
        se_vals[plain_key] = std_err_plain
    
    # Build MultiIndex columns
    columns = pd.MultiIndex.from_product(
        [maturation, methods],
        names=["Maturation", ""]
    )

    # Collect rows
    rows = {
        "True value": [theoretical[(str(T), m)] for T in maturation for m in methods],
        "Estimate":   [mc_est[(str(T), m)] for T in maturation for m in methods],
        "S.E.":       [se_vals[(str(T), m)] for T in maturation for m in methods],
    }

    # Create DataFrame
    df = pd.DataFrame(rows, index=columns).T

    return df

In [19]:
results_6_3 = example_6_3()
results_6_3

Maturation     1.0000            0.5000            0.2500       
           antithetic  plain antithetic  plain antithetic  plain
True value     0.9192            1.3138            1.8156       
Estimate       0.9172 0.9126     1.3194 1.3075     1.8434 1.8198
S.E.           0.0122 0.0102     0.0152 0.0115     0.0166 0.0120

## Control Variates

**Example 6.4. Underlying stock price as control variate.** Use the method of control
variates to estimate the price of a call option with maturity $T$ and strike price
$K$. Since $$ E[e^{-r T} S_T] = S_0, $$ the discounted stock price $e^{-r T} S_T$
can serve as a control variate.

In [20]:
def example_6_4():
    rng = np.random.default_rng(4198)
    
    # simulation params
    S_0, r, sig, T = 50, 0.05, 0.2, 1
    n = 10_000
    b = 1

    def plain(K):
        # sampling
        Z = norm.rvs(0, 1, size=n, random_state=rng)
        Y = S_0 * np.exp((r - 0.5 * sig**2) * T + sig * np.sqrt(T) * Z)
        X = np.exp(-r * T) * np.maximum(Y - K, 0)

        # compute mean est. and std. error
        v_hat = np.mean(X)
        se = sem(X)
        re = se / v_hat

        return v_hat, se, re
    
    def control_variate_discounted_stock_price(K):
        # sampling
        Z = norm.rvs(0, 1, size=n, random_state=rng)
        S = S_0 * np.exp((r - 0.5 * sig**2) * T + sig * np.sqrt(T) * Z)
        X = np.exp(-r * T) * np.maximum(S - K, 0)
        Y = np.exp(-r * T) * S - S_0

        H = X - b * Y

        # compute mean est. and std. error
        v_hat = np.mean(H)
        se = sem(H)
        re = se / v_hat

        # b_opt_est used instead of b
        b_opt_est = np.cov(X, Y)[0, 1] / np.var(Y)
        H_b_opt_est = X - b_opt_est * Y

        v_hat_b_opt_est = np.mean(H_b_opt_est)
        se_b_opt_est = sem(H_b_opt_est)
        re_b_opt_est = se_b_opt_est / v_hat_b_opt_est 

        return v_hat, se, re, v_hat_b_opt_est, se_b_opt_est, re_b_opt_est
    
    mc_est, se_vals, re_vals, theoretical = {}, {}, {}, {}
    strikes, methods = [45, 50, 55, 60, 65], ["b", "b_opt_est", "plain"]
    
    for K in strikes:
        theoretical_value = european_call(S_0, K, r, T, sig)
        mean_est_plain, std_err_plain, re = plain(K)
        mean_est_b, std_err_b, re_b, mean_est_b_opt_est, std_err_b_opt_est, re_b_opt_est = control_variate_discounted_stock_price(K)

        b_key = (f"{K}", "b")
        b_opt_est_key = (f"{K}", "b_opt_est")
        plain_key = (f"{K}", "plain")

        theoretical[b_key] = ""
        mc_est[b_key] = mean_est_b
        se_vals[b_key] = std_err_b
        re_vals[b_key] = re_b

        theoretical[b_opt_est_key] = theoretical_value
        mc_est[b_opt_est_key] = mean_est_b_opt_est
        se_vals[b_opt_est_key] = std_err_b_opt_est
        re_vals[b_opt_est_key] = re_b_opt_est

        theoretical[plain_key] = ""
        mc_est[plain_key] = mean_est_plain
        se_vals[plain_key] = std_err_plain
        re_vals[plain_key] = re

    
    # Build MultiIndex columns
    columns = pd.MultiIndex.from_product(
        [strikes, methods],
        names=["Strike price", ""]
    )

    # Collect rows
    rows = {
        "True value": [theoretical[(str(K), m)] for K in strikes for m in methods],
        "Estimate":   [mc_est[(str(K), m)] for K in strikes for m in methods],
        "S.E.":       [se_vals[(str(K), m)] for K in strikes for m in methods],
        "R.E.":       [re_vals[(str(K), m)] for K in strikes for m in methods],
    }

    # Create DataFrame
    df = pd.DataFrame(rows, index=columns).T
    
    return df

In [21]:
results_6_4 = example_6_4()
results_6_4

Strike price     45                      50                      55            \
                  b b_opt_est  plain      b b_opt_est  plain      b b_opt_est   
True value             8.3497                  5.2253                  3.0200   
Estimate     8.3239    8.3470 8.2868 5.2774    5.2593 5.1924 2.8824    2.9564   
S.E.         0.0267    0.0211 0.0865 0.0439    0.0283 0.0727 0.0590    0.0301   
R.E.         0.0032    0.0025 0.0104 0.0083    0.0054 0.0140 0.0205    0.0102   

Strike price            60                      65                   
              plain      b b_opt_est  plain      b b_opt_est  plain  
True value                    1.6237                  0.8198         
Estimate     2.9962 1.5977    1.5984 1.5532 0.8199    0.8091 0.8348  
S.E.         0.0571 0.0736    0.0281 0.0418 0.0847    0.0241 0.0329  
R.E.         0.0191 0.0461    0.0175 0.0269 0.1033    0.0298 0.0394

**Example 6.5. Analytically tractable derivatives as control variate.**  Estimate
the price of a discretely monitored average price call option with maturity $T$
and payoff $(\bar{S} − K)^+$. Here $\bar{S}$ is the arithmetic mean of stock prices:
$$\bar{S} = \frac{1}{m} \sum_{k = 1}^{m} S_{t_k},$$
where $0 < t_1 < t_2 < \cdots < t_m = T$ are given dates. Use the average price call
option with geometric mean as the control variate.

In [37]:
def example_6_5():
    rng = np.random.default_rng(4198)
    
    # simulation params
    S_0, r, sig, T = 50, 0.05, 0.2, 1
    m, n = 12, 10_000
    t = [(i+1)/m for i in range(m)]
    b = 1

    def plain(K):
        # sampling
        Z = norm.rvs(0, 1, size=(m, n), random_state=rng)
        St = np.zeros(shape=(m, n))

        t_diff = [t[0]] + [t[i] - t[i-1] for i in range(1, m)]
        
        St[0] = S_0 * np.exp((r - 1/2 * sig**2) * t_diff[0] + sig * np.sqrt(t_diff[0]) * Z[0])
        for i in range(1, m):
            St[i] = St[i-1] * np.exp((r - 1/2 * sig**2) * t_diff[i] + sig * np.sqrt(t_diff[i]) * Z[i])
        
        X = np.exp(-r*T) * np.maximum(np.mean(St, axis=0) - K, 0)

        # compute mean est. and std. error
        v_hat = np.mean(X)
        se = sem(X)

        return v_hat, se
    
    def control_variate_geometric_average_price_call(K):
        # sampling
        Z = norm.rvs(0, 1, size=(m, n), random_state=rng)
        St = np.zeros(shape=(m, n))

        t_diff = [t[0]] + [t[i] - t[i-1] for i in range(1, m)]
        
        St[0] = S_0 * np.exp((r - 1/2 * sig**2) * t_diff[0] + sig * np.sqrt(t_diff[0]) * Z[0])
        for i in range(1, m):
            St[i] = St[i-1] * np.exp((r - 1/2 * sig**2) * t_diff[i] + sig * np.sqrt(t_diff[i]) * Z[i])
        
        S_bar = np.mean(St, axis=0)
        S_bar_G = np.exp(np.log(St).mean(axis=0))
        p = discretely_monitored_average_price_call_option(S_0, K, r, T, sig, m)

        X = np.exp(-r*T) * np.maximum(S_bar - K, 0)
        Y = np.exp(-r*T) * np.maximum(S_bar_G - K, 0) - p

        H = X - b * Y

        # compute mean est. and std. error
        v_hat = np.mean(H)
        se = sem(H)

        # b_opt_est used instead of b
        b_opt_est = np.cov(X, Y)[0, 1] / np.var(Y)
        H_b_opt_est = X - b_opt_est * Y

        v_hat_b_opt_est = np.mean(H_b_opt_est)
        se_b_opt_est = sem(H_b_opt_est)

        return v_hat, se, v_hat_b_opt_est, se_b_opt_est
    
    mc_est, se_vals = {}, {}
    strikes, methods = [45, 55], ["b", "b_opt_est", "plain"]
    
    for K in strikes:
        mean_est_plain, std_err_plain = plain(K)
        mean_est_b, std_err_b, mean_est_b_opt_est, std_err_b_opt_est = control_variate_geometric_average_price_call(K)

        b_key = (f"{K}", "b")
        b_opt_est_key = (f"{K}", "b_opt_est")
        plain_key = (f"{K}", "plain")

        mc_est[b_key] = mean_est_b
        se_vals[b_key] = std_err_b

        mc_est[b_opt_est_key] = mean_est_b_opt_est
        se_vals[b_opt_est_key] = std_err_b_opt_est

        mc_est[plain_key] = mean_est_plain
        se_vals[plain_key] = std_err_plain
    
    # Build MultiIndex columns
    columns = pd.MultiIndex.from_product(
        [strikes, methods],
        names=["Strike price", ""]
    )

    # Collect rows
    rows = {
        "Estimate":   [mc_est[(str(K), m)] for K in strikes for m in methods],
        "S.E.":       [se_vals[(str(K), m)] for K in strikes for m in methods],
    }

    # Create DataFrame
    df = pd.DataFrame(rows, index=columns).T
    
    return df

In [38]:
results_6_5 = example_6_5()
results_6_5

Strike price     45                      55                 
                  b b_opt_est  plain      b b_opt_est  plain
Estimate     6.4585    6.4588 6.4459 1.1465    1.1464 1.1467
S.E.         0.0017    0.0014 0.0546 0.0018    0.0011 0.0276

## Stratified Sampling

**Example 6.6** Assume that under the risk-neutral probability measure the
stock price is a geometric Brownian motion
$$ S_t = S_0 \exp \left\{ \left( r - \frac{1}{2} \sigma^2 \right) t - \sigma W_t \right\}. $$
Design a stratified sampling scheme to estimate the price of a call option with
maturity $T$ and strike price $K$.

In [25]:
def basic_stratified_sampling(n, k, q, F_inv, h):
    rng = np.random.default_rng(4198)

    nn = n * q
    nn = nn.astype(int)

    mu_hat = np.zeros(k)
    var = np.zeros(k)
    for i in range(k):
        V = uniform.rvs(size=nn[i], random_state=rng)
        U = i/k + V/k
        X = h(F_inv(U))
        
        mu_hat[i] = np.mean(X)
        var[i] = np.var(X, ddof=1)
    
    v_hat = 1/k * np.sum(mu_hat)
    se = 1/k * np.sqrt(np.sum(var/nn))

    return v_hat, se

In [26]:
def proportional_allocation_stratified_sampling(n, k, F_inv, h):
    q = np.array([1/k] * k)
    return basic_stratified_sampling(n, k, q, F_inv, h)

In [27]:
def example_6_6():
    # simulation params
    S_0, r, sig, T = 50, 0.05, 0.2, 1
    n = 10_000

    def stratified_sampling(k, K):
        h = lambda x: np.maximum(S_0 * np.exp(-1/2 * sig**2 * T + sig * np.sqrt(T)*x) - np.exp(-r*T) * K, 0)
        F_inv = norm.ppf
        return proportional_allocation_stratified_sampling(n, k, F_inv, h)

    mc_est, se_vals, theoretical_vals = {}, {}, {}
    k, K = [25, 100], [40, 50, 60]
    
    for ki in k:
        for Ki in K:
            theoretical_val = european_call(S_0, Ki, r, T, sig)
            mean_est, std_err = stratified_sampling(ki, Ki)

            key = (Ki, ki)

            theoretical_vals[key] = theoretical_val if ki == 25 else ""
            mc_est[key] = mean_est
            se_vals[key] = std_err

        
    # Build MultiIndex columns
    columns = pd.MultiIndex.from_product(
        [K, k],
        names=["Strike price", "# of strata"]
    )

    # Collect rows
    rows = {
        "True value": [theoretical_vals[(Ki, ki)] for Ki in K for ki in k],
        "Estimate":   [mc_est[(Ki, ki)] for Ki in K for ki in k],
        "S.E.":       [se_vals[(Ki, ki)] for Ki in K for ki in k],
    }

    # Create DataFrame
    df = pd.DataFrame(rows, index=columns).T

    return df

In [28]:
results_6_6 = example_6_6()
results_6_6

Strike price      40             50            60       
# of strata      25      100    25     100    25     100
True value   12.2944         5.2253        1.6237       
Estimate     12.2749 12.2895 5.2058 5.2205 1.6054 1.6192
S.E.          0.0115  0.0054 0.0113 0.0054 0.0112 0.0054

**Example 6.7.** Use stratified sampling to estimate the price of a spread call
option with maturity $T$ and payoff
$$(X_T - Y_T - K)^+,$$
where $X$ and $Y$ are prices of two underlying assets. Assume that under the
risk-neutral probability measure,
$$X_t = X_0 \exp \left\{ \left( r - \frac{1}{2} \sigma_1^2 \right) t - \sigma_1 W_t \right\}, $$
$$Y_t = Y_0 \exp \left\{ \left( r - \frac{1}{2} \sigma_2^2 \right) t - \sigma_2 B_t \right\}, $$
where $(W, B)$ is a two-dimensional Brownian motion with covariance matrix
$$ \Sigma = \begin{bmatrix} 1 & \rho \\ \rho & 1 \end{bmatrix}.$$

In [29]:
def example_6_7a():
    # stratifying theta alone
    X_0, Y_0 = 50, 45
    r, sig1, sig2, T = 0.05, 0.2, 0.3, 1
    rho = 0.5

    n = 10_000
    
    def stratified_sampling(k, K):
        def h(theta):
            eta = norm.rvs(loc=rho * theta, scale = np.sqrt(1 - rho**2), size=len(theta))

            X = X_0 * np.exp((r - 1/2 * sig1**2) * T + sig1 * np.sqrt(T) * theta)
            Y = Y_0 * np.exp((r - 1/2 * sig2**2) * T + sig2 * np.sqrt(T) * eta)

            return np.exp(-r*T) * np.maximum(X - Y - K, 0)


        F_inv = norm.ppf

        return proportional_allocation_stratified_sampling(n, k, F_inv, h)

    mc_est, se_vals = {}, {}
    k, K = [25, 100], [0, 5, 10]
    
    for ki in k:
        for Ki in K:
            mean_est, std_err = stratified_sampling(ki, Ki)

            key = (Ki, ki)
            mc_est[key] = mean_est
            se_vals[key] = std_err

        
    # Build MultiIndex columns
    columns = pd.MultiIndex.from_product(
        [K, k],
        names=["Strike price", "# of strata"]
    )

    # Collect rows
    rows = {
        "Estimate":   [mc_est[(Ki, ki)] for Ki in K for ki in k],
        "S.E.":       [se_vals[(Ki, ki)] for Ki in K for ki in k],
    }

    # Create DataFrame
    df = pd.DataFrame(rows, index=columns).T
    return df

In [30]:
results_6_7a = example_6_7a()
results_6_7a

Strike price     0             5             10       
# of strata     25     100    25     100    25     100
Estimate     7.9113 7.8684 4.9026 4.9099 2.7815 2.8047
S.E.         0.0779 0.0777 0.0632 0.0623 0.0469 0.0474

In [31]:
def example_6_7b():
    # stratifying both theta and eta

    rng = np.random.default_rng(1)

    X_0, Y_0 = 50, 45
    r, sig1, sig2, T = 0.05, 0.2, 0.3, 1
    rho = 0.5

    n = 10_000
    
    def stratified_sampling(k, K):
        kk = k**2
        nn = n // kk
        C = np.array([
            [1, 0],
            [rho, np.sqrt(1 - rho**2)]
        ])


        mu_hat = np.zeros(shape=(k, k))
        var = np.zeros(shape=(k, k))
        for i in range(k):
            for j in range(k):
                U = i / k + uniform.rvs(size=nn) / k
                V = j / k + uniform.rvs(size=nn) / k
                Z = np.vstack((norm.ppf(U), norm.ppf(V)))

                theta, eta = C @ Z
                X = X_0 * np.exp((r - 1/2 * sig1**2) * T + sig1 * np.sqrt(T) * theta)
                Y = Y_0 * np.exp((r - 1/2 * sig2**2) * T + sig2 * np.sqrt(T) * eta)

                H = np.exp(-r * T) * np.maximum(X - Y - K, 0)
                mu_hat[i][j] = np.mean(H)
                var[i][j] = np.var(H, ddof=1)
        
        v_hat = mu_hat.mean()
        se =  np.sqrt(np.sum(var) / nn) / kk
        return v_hat, se
    
    mc_est, se_vals = {}, {}
    k, K = [5, 10], [0, 5, 10]
    
    for ki in k:
        for Ki in K:
            mean_est, std_err = stratified_sampling(ki, Ki)

            key = (Ki, ki)
            mc_est[key] = mean_est
            se_vals[key] = std_err

        
    # Build MultiIndex columns
    columns = pd.MultiIndex.from_product(
        [K, k],
        names=["Strike price", "# of strata k^2"]
    )

    # Collect rows
    rows = {
        "Estimate":   [mc_est[(Ki, ki)] for Ki in K for ki in k],
        "S.E.":       [se_vals[(Ki, ki)] for Ki in K for ki in k],
    }

    # Create DataFrame
    df = pd.DataFrame(rows, index=columns).T
    return df


In [32]:
results_6_7b = example_6_7b()
results_6_7b

Strike price        0             5             10       
# of strata k^2     5      10     5      10     5      10
Estimate        7.8308 7.9042 4.9696 4.9653 2.7878 2.7904
S.E.            0.0232 0.0148 0.0227 0.0137 0.0208 0.0132